# Lab 4 Part II: Radio Communication via an RPiTX and the SDR

This is an alpha version of the lab, meant for those who have issues with their radio or audio interface, or have not gotten a radio for a number of reasons. I developed this lab over Spring break 2020, being sheltered in place due to the COVID-19 virus epidemic. 

Again, this is an *alpha* version and has not been debugged almost at all -- so things may not work, break or vary for different people. Please bear that in mind and work with us to perfect this lab.

In this lab you will receive RF signals using the RTL-SDR, and transmit signals using the Raspberry Pi itself. This lab heavily uses several amazing open source projects, which I'm grateful for. 

## Turning the Raspberry Pi into a transmitter
The original work for this hack was created by the Imperial College Robotic Society. It was later extended by Oliver Mattos and Oskar Weigle to the PiFM code, and finally turned into a full-fledged package called RPiTX by Evariste Courjad (F50EO). 

The hack uses the hardware on the Raspberry Pi that is actually meant to generate variable frequencies -- computer controlled clock signals on the GPIO pins. By cleverly using DMA and inputting different frequencies at different times, the software can generate a frequency modulated (FM) signal out of GPIO 4 (pin 7) of the Raspberry Pi.
Any freqency between 0 and 250 MHz can be set, and because the carrier signal is a square wave, the signal produced has very strong harmonics. By using the harmonics it is possible to generate carriers even at 1 GHz. 

When connected to an antenna or a wire, this "radio" will result in illegal emmissions (harmonics). But without any wire or antenna connected to the pins, the amount of power emmitted is small and ranges over a few feet. This will enable us to use this approach for this lab, without implementing a bandpass filter. 

This code, along with RTL-SDR and CSDR, are the basis for making this lab work. 

### References:

* [ICRobotics](http://www.icrobotics.co.uk/wiki/index.php/Turning_the_Raspberry_Pi_Into_an_FM_Transmitter)
* [PiFM github](https://github.com/rm-hull/pifm)
* [RPiTX github](https://github.com/F5OEO/rpitx)
* [librtlsdr](https://github.com/librtlsdr/librtlsdr)
* [CSDR](https://github.com/ha7ilm/csdr)

## Hardware Setup

For this lab, you will need **your Raspberry Pi, speaker, SDR and antenna.**

At various points in the lab, you may need to move your antenna around to adjust performance. Keep in mind that you do not want the antenna very far away from the SDR due to the low power output, and you do not want the antenna too close to the other equipment due to interference.

Your setup should look like the following figure.

<center><img src="./pinout.jpg" alt="gsm" style="width: 400px;"/></center>
<center>Figure 1: Raspberry Pi pinout</center>

<center><img src="./rpitx-setup.jpg" alt="gsm" style="width: 400px;"/></center>
<center>Figure 2: Equipment setup</center>


## The Radio class (wrapping rtlsdr and rpitx in python)

The way I implemented the "radio" is by calling shell commands that use rtl_fm (a software for demodulating FM radio signal using rtlsdr), rpitx, and CSDR. To pipe the audio into and out of python, we leverage the loopback audio device, which is a virtual device enabling piping of audio between applications. 

In order for this to work, you need to have snd_aloop linux kernel module loaded. You also need to be able to run sudo without password entry. 

Make sure you do the following: open a terminal on the Pi through SSH, bluetooth, or serial connection. 

1. In the terminal, type:
~~~~
sudo visudo
~~~~

Append the following line to the end of the file, and save:
~~~~
pi ALL=(ALL) NOPASSWD:ALL
~~~~

You should now be able to run `sudo` without a password. 

2. In the terminal, type:
~~~~
sudo modprobe snd_aloop
~~~~

This will load the loopback audio module. **NOTE: Step 2 is not persistent, and you will need to do this every time you boot the Pi.** 

Two important audio devices that are created are: `plughw:CARD=Loopback,DEV=0`,  and `plughw:CARD=Loopback,DEV=1`. These are the loopback devices. We will use `DEV=1` from python and `DEV=0` from outside.

The following code should show if the Loopback module was loaded and the devices are set. You should see the output
~~~~
plughw:CARD=Loopback,DEV=0
plughw:CARD=Loopback,DEV=1
~~~~
if it worked.

In [ ]:
! sudo modprobe snd_aloop

In [ ]:
# the following code should show if the Loopback module was loaded and the devices are set.

! aplay -L | grep plughw:CARD=Loopback,DEV=

3. Because of some HW interference issues, the rpitx provided in your installation will have hardware collisions with the sound interface. To minimize this, we patched rpitx and provided you with an executable. You need to copy that executable to /usr/bin/ by:

```
sudo cp rpitx /usr/bin
```


In [ ]:
! sudo cp rpitx /usr/bin/

Import the Radio class, spectrogram, and spectrum display from 'lab4_2_utils.py'

In [ ]:
from lab4_2_utils import *

In [ ]:
# import functions and libraries

import numpy as np
import matplotlib.pyplot as plt
import pyaudio, threading, time, sys, serial
import queue as Queue
from numpy import pi, sin, zeros, r_
from scipy import signal
from rtlsdr import RtlSdr
import sounddevice as sd
%matplotlib inline

In [ ]:
# set up alsamixer volumes for the Raspberry Pi

!amixer -c 1 -- sset 'Capture Mux' 'LINE_IN'
!amixer -c 1 -- sset Lineout playback unmute
!amixer -c 1 -- sset Lineout playback 50%,50%
!amixer -c 1 -- sset Mic capture 66%
!amixer -c 1 -- sset Mic playback 66%

## Testing audio I/O and radio receive

### Testing the audio:

The first test/example would be to see if we can capture audio from the radio and play it on the Raspberry Pi.

An important feature in this task is that **you will have a method to check if the incoming signal is being clipped. Remember the settings for which the signal maximum is 0.6. This would be VERY useful in the communications part of the lab.** This is less of a problem for the sdr + rpitx setup than the audio inteface and the Baofeng, but still...

Another important caveat: due to intefering devices within the Pi, it is not possible to transmit with rpitx and *also* play sound using any of the sound interfaces. Any attempt to do so will require a restart. Don't say I did not warn you!!!

Let's start a radio object:

In [ ]:
R = Radio()

Let's set the frequency to a local FM station. At Berkeley, it would be 94.1 MHz, or 94100 kHz. We will also route the radio output directly to the Fe-Pi audio. **Make sure your speaker is connected to the RX/Speaker green connector.** Make sure the SDR gain is set appropriately (gain of 36 is reasonable).  

In [ ]:
R.audiodev_out = "plughw:CARD=Audio,DEV=0"
R.freq_rx = 94100.0
R.sdr_gain = 36

We are ready to run the wide-band FM receiver. After running, you will see the shell command line that was used. 

In [ ]:
R.WBFMrx()

To stop the device we run R.stoprx(), or R.close(). The latter will also kill any transmitter. 

In [ ]:
# R.stoprx()
R.close()

We would like to run the radio while piping the audio to the loopback audio device, so we can read it within python. We will first need to set the audio. We will be using the SoundDevice python audio library. 

The following command will list the audio devices on your pi. You should see:
```
 0 bcm2835 ALSA: - (hw:0,0), ALSA (0 in, 2 out)
  1 bcm2835 ALSA: IEC958/HDMI (hw:0,1), ALSA (0 in, 2 out)
  2 Fe-Pi Audio: - (hw:1,0), ALSA (2 in, 2 out)
  3 Loopback: PCM (hw:2,0), ALSA (32 in, 32 out)
  4 Loopback: PCM (hw:2,1), ALSA (32 in, 32 out)
  5 sysdefault, ALSA (0 in, 128 out)
  6 dmix, ALSA (0 in, 2 out)
* 7 default, ALSA (2 in, 2 out)
```
You may see something slightly different than the above. 
But, if you don't see a loopback, you need to repeat the `sudo modprobe snd_aloop`. If you don't see the Fe-Pi Audio device, then it means that something is off. Try rebooting! 

In [ ]:
sd.query_devices()

Manually set the audio device numbers. 

builtin_idx = 2 #should be the Fe-Pi Audio

loop1_idx  = 4 #should be Loopback: PCM (hw:2,1)


In [ ]:
builtin_idx = 2 # should be the Fe-Pi Audio
loop1_idx = 4 # should be Loopback: PCM (hw:2,1)

We will also set up default of 1 channel and sampling rate of 48 kHz:

In [ ]:
# set default sample rate and number of channels. 
sd.default.samplerate = 48000
sd.default.channels = 1

The code below shows how to read data from a device, using a callback function and a stream. The callback function in this case is simple: it copies the input data stream (loop1) to an output data stream (builtin), so we can hear what is being streamed by the radio.

In [ ]:
# this callback function will play captured data 
# it will be called by the sounddevice stream and run in a different thread

def replay_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    outdata[:] = indata  # the [:] is important so data is copied not referenced

Let's set up a new radio. Since the default device when starting the radio is the loopback, we can just create a new one and set frequencies and gains. Then run the WBFM receiver again and start receiving FM radio.

In [ ]:
R = Radio()
R.freq_rx = 94100.0
R.sdr_gain = 36
# R.audiodev_out = "plughw:CARD=Loopback,DEV=0" # optional, since it is a default.
R.WBFMrx()

In [ ]:
# create stream
# will record from device "loop1" and play through device "builtin" 
st = sd.Stream(device=(loop1_idx, builtin_idx), callback=replay_callback)

# start stream -- will run in background till stopped
st.start()

In [ ]:
# stop and close stream -- must stop and close for clean exit
st.stop()
st.close()
R.stoprx()

You may need to change the volume of the audio by setting R.audio_rcv_gain, which can be any positive floating-point number.

In [ ]:
R = Radio()
R.freq_rx = 94100.0
R.sdr_gain = 36
# R.audiodev_out = "plughw:CARD=Loopback,DEV=0" # optional, since it is a default.
R.audio_rcv_gain = 0.1
R.WBFMrx()

# create stream
# will record from device "loop1" and play through device "builtin"
st = sd.Stream(device=(loop1_idx, builtin_idx), callback=replay_callback)

# start stream -- will run in background till stopped
st.start()

In [ ]:
# stop and close stream -- must stop and close for clean exit
st.stop()
st.close()
R.stoprx()

The following callback will do exactly the same thing as before. The only difference is that the received audio will be pushed to a Queue so we can process it outside of the callback function. Default block-size is 512 samples, which is about 10 ms worth of samples -- this seems to be a problem for the Pi. We will therefore use 1024 samples per block, which is about 20 ms worth of samples.

We will capture just over 10 seconds, which is about 500 blocks. The samples from the queue will be processed. We will compute the maximum signal and the root-mean-square (RMS) for each block. This will let us see if the signal is being clipped. 

In [ ]:
def queuereplay_callback(indata, outdata, frames, time, status):
    assert frames == 1024
    if status:
        print(status, file=sys.stderr)
    outdata[:] = indata # keep this only if you are not transmitting!
    Qin.put( indata.copy() ) # global queue

In [ ]:
R = Radio()
R.freq_rx = 94100.0
R.sdr_gain = 36
# R.audiodev_out = "plughw:CARD=Loopback,DEV=0" # optional, since it is a default.
R.audio_rcv_gain = 1.5
R.WBFMrx()

In [ ]:
# create an input FIFO queue
Qin = Queue.Queue()

st = sd.Stream(device=(loop1_idx, builtin_idx), blocksize=1024, callback=queuereplay_callback)

st.start()

# record and play about 10.6 seconds of audio 500*1024/48000 = 10.6 s
mxpwr = zeros(500)
rmspwr = zeros(500)

for n in range(500):
    samples = Qin.get()
    mxpwr[n] = max(abs(samples))
    rmspwr[n] = np.sqrt(np.sum(np.square(samples)))
    # You can add code here to do processing on samples in chunks of 512 samples
    # In general, you will have to implement an overlap and add, or overlap an save to get
    # continuity between chunks -- we will do this later!

st.stop()
st.close()

# empty queue just in case there's something left
while not(Qin.empty()):
    samples = Qin.get()
R.stoprx()

The code also displays the RMS amplitude and maximum audio signal for each 1024-sample block -- so you can see if it is clipped or too weak.

In [ ]:
fig = plt.figure(figsize=(16,4))
t = r_[0:500]*1024/48000
plt.plot(t, mxpwr)
plt.plot(t, rmspwr/np.sqrt(1024))
plt.title('Maximum/RMS amplitude')
plt.legend(('Max amplitude','RMS amplitude'))

if any(mxpwr > 0.95):
    print("Warning! Signal is clipped. Reduce radio volume, and/or usb device input volume.")
if max(mxpwr) < 0.3:
    print("Audio volume may be too low. Increase the volume on the radio for better lab performance.")

Make sure you set the volume such that the peak is not higher than 0.8.

The "radio" is also able to demodulate narrow-band FM, such as Walkie Talkies, NOAA weather, and the Baofeng radio.
Pick a local NOAA weather repeater frequency (162.400 MHz, 162.425 MHz, 162.450 MHz, 162.475 MHz, 162.500 MHz, 162.525 MHz, and 162.550 MHz), or if you have a Baofeng radio and a license, choose an amateur radio band (one of the experimental frequencies programmed on your radio) and test the receiver. Repeat the recording and amplitude experiment from above with narrowband FM. You can transmit while pressing on the keys to play DTMF tones.

The code: ``R.NFMrx()`` starts a narrowband FM receiver. 

In [ ]:
R = Radio()
R.freq_rx = 162425.0
R.sdr_gain = 36
# R.audiodev_out = "plughw:CARD=Loopback,DEV=0" # optional, since it is a default.
R.audio_rcv_gain = 1
R.NFMrx()

# create an input FIFO queue
Qin = Queue.Queue()

st = sd.Stream(device=(loop1_idx, builtin_idx), blocksize=1024, callback=queuereplay_callback)

st.start()

# record and play about 10.6 seconds of audio 500*1024/48000 = 10.6 s
mxpwr = zeros(500)
rmspwr = zeros(500)

for n in range(500):
    samples = Qin.get()
    mxpwr[n] = max(abs(samples))
    rmspwr[n] = np.sqrt(np.sum(np.square(samples)))
    # You can add code here to do processing on samples in chunks of 512 samples
    # In general, you will have to implement an overlap and add, or overlap an save to get
    # continuity between chunks -- we will do this later!

st.stop()
st.close()

# empty queue just in case there's something left
while not(Qin.empty()):
    samples = Qin.get()

R.stoprx()

fig = plt.figure(figsize=(16,4))
t = r_[0:500]*1024/48000
plt.plot(t, mxpwr)
plt.plot(t, rmspwr/np.sqrt(1024))
plt.title('Maximum/RMS amplitude')
plt.legend(('Max amplitude','RMS amplitude'))

if any(mxpwr > 0.95):
    print("Warning! Signal is clipped. Reduce radio volume, and/or usb device input volume.")
if max(mxpwr) < 0.3:
    print("Audio volume may be too low. Increase the volume on the radio for better lab performance.")

# Testing radio transmission

The next step is to test the transmission through RPiTX. 
If you have a Baofeng radio and for some reason you cannot do the "interface" (old) version of this lab, you can use it to listen to the transmission and validate that things are working. For this case, I recommennd using one of the UHF experimental channels 71-98 on your radio. 

If you do not have a radio, then I recommend using the unlicensed band 919 MHz. Do not connect any wire to GPIO 4 (pin 7), on the Pi, so that you do not violate any FCC rules.

The basics for transmitting with RPiTX are similar to the receive, in term of configuration.

The important variables are:
~~~~
Radio.freq_tx = transmit_freq 
Radio.audiodev_in="hw:CARD=ALSA,DEV=0"
Radio.NFMtx()
Radio.stoptx() # or Radio.close()
~~~~

## Transmitting audio from the Pi to the radio

Below is code that:
- transmits a 2 kHz tone for 2 seconds
- pauses for a little while
- transmits a 1 kHz tone for 2 seconds
- receives the transmission and records it
- plays back the recording

Unfortunately, we cannot transmit, receive, and play at the same time. 
Instead, we will transmit RF via GPIO 4 (pin 7), receive with the SDR, and push the received audio to the loopback device. 
At the same time, we will use a streaming audio interface to read loopback samples and put in a Queue. 
After finishing the transmission and reception, we will terminate the transmit and play the sound from the Queue.

Sounds good?

Here's a callback function which only stores samples in a queue:

In [ ]:
def queuerecord_callback(indata, frames, time, status):
    if status:
        print(status)
    Qin.put( indata.copy() ) # global queue

Create a receiver and start a stream for saving the receiver output.

In [ ]:
# create a receiver
R = Radio()
R.freq_rx = 919000.0
R.sdr_gain = 50
# R.audiodev_out = "plughw:CARD=Loopback,DEV=0" # optional, since it is a default.

# start receiving
R.NFMrx()

# start the queue for receiving
Qin = Queue.Queue()
st = sd.InputStream(device=loop1_idx, blocksize=1024, callback=queuerecord_callback)
st.start()

Perform the transmission.

In [ ]:
# choose transmission frequency:
R.freq_tx = 919000.0

# generate sinusoids
fs_se = 48000 # audio sampling rate
t = r_[0.0:fs_se*2]/fs_se
sig1 = 0.5*sin(2*pi*2e3*t)
sig2 = 0.5*sin(2*pi*1e3*t)

R.NFMtx()
time.sleep(0.1) # give radio time to start
# play audio on the sound extension. When blocking is True, this will run in the foreground.
sd.play(sig1, samplerate=fs_se, device=loop1_idx, blocking=True)

R.stoptx()
time.sleep(0.5)
R.NFMtx()
time.sleep(0.1) # give radio time to start
sd.play(sig2, samplerate=fs_se, device=loop1_idx, blocking=True)

R.close()
st.stop()
st.close()

Assemble the received samples into a single array and play it on your speaker. 

In [ ]:
if not(Qin.empty()):
    recorded_radio = Qin.get_nowait()
while not(Qin.empty()):
    recorded_radio = np.concatenate((recorded_radio, Qin.get_nowait()))

In [ ]:
sd.play(recorded_radio, samplerate=fs_se, device=builtin_idx, blocking=True)

### Calibrating the input audio level to the radio using radio transmission and reception with the SDR 

A few facts about handheld FM radios:

In general, the audio input to the radio is filtered within the radio by a bandpass filter, which passes frequencies roughly between 500 Hz and 4 kHz. The input filter also emphasizes the high frequencies with approximately 6 dB per decade up to 3 kHz. This is because FM has higher noise in the high frequency, so by sending higher amplitude for high frequencies, the SNR remains the same for all frequencies.

Another fact about FM is that the amount of frequency deviation is proportional to the amplitude of the audio. However, before transmitting, the modulated FM signal goes through a bandpass filter so that energy does not leak to other channels. If the input volume is too high, the output bandpass filter will "crop" the signal and the transmitted audio will be distorted. It is therefore important that we have a way to set the right level of output such that there's no clipping of the signal. 

In this task, we will transmit a pure audio tone with linearly increasing amplitude. We will receive the signal using the SDR and FM demodulate it (similarly to Lab 3). We will then determine the amplitude in which the signal is still "well behaved" -- not clipped and without non-linearities.

Here's how it works with physical radios: 
* We generate a tone with increasing amplitude in python.
* The audio interface converts it into an analog signal -- acting as a DAC.
* The radio filters the input with its audio bandpass filter.
* The radio FM modulates the signal with $\pm7.5$ kHz deviation at the chosen center frequency and transmits the FM signal.

Here's how we will simulate it with RPiTX:
* We will generate a tone with increasing amplitude in python.
* We will "play" the tone to the virtual loopback audio device, which will pass it to the RPiTX script.
* RPiTX script will read the "audio" and use CSDR to perform digital bandpass filtering.
* CSDR will then perform the FM modulation and provide RPiTX with the appropriate output.
* RPiTX will transmit the signal at the chosen center frequency. 

Here's how the receiving will work:
* The SDR will capture samples around that center frequency.
* You will implement a carrier squelch that will crop the samples corresponding to the transmission.
* You will FM demodulate the signal by implementing a lowpass filter, limiter, and discriminator similarly to Lab 3. 
* You will then look at the amplitude of the received tone. The amplitude should increase linearly at first, then taper off and saturate. You need to figure out which audio amplitude values are in the linear regime. The range of values you will get will correspond to the range of audio signals that will not be distorted using your audio settings!

The latter part is only necessary when using a physical radio, where the gains of your Raspberry Pi and the input amplifier of the radio are not known. Here, any signal between 0-1.0 should be linear -- but to show the effect, we will drive it beyond. 

#### Pre-Task:  Setting the gain of the SDR and calibrating the frequency

It is important to know that even though we set a transmit and receive frequency, the frequency the radio is transmitting and the frequency the SDR is receiving may not be exactly the same. This is because the crystal oscillator on the Raspberry Pi is rated to about 1 ppm, and the SDR is rated to about 70 ppm deviation.  
Before we start, we would like to make sure that the SDR frequency is calibrated to the radio (both may have some offset). We would also like to adjust the gain of the SDR, so it is not under/overdriven by the radio.

For this, we will transmit a carrier at the center frequency of the radio and receive at the center frequency of the SDR. We will look at the spectrum to see the offset between the transmitted frequency and the received one. We will then calibrate the offset of the SDR with respect to the radio. The SDR has a parameter "ppm" for prescribing known frequency offsets.

We will also look at the received amplitude to see if it's clipped.

* Acquire 2 seconds of data while the radio is transmitting.
* Plot the amplitude of the signal. Make sure the amplitude of the signal is > 0.25 and < 0.75. If not, change the gain of the SDR or move the receive antenna away from the radio.
* Plot the average power spectrum or the spectrogram, and calculate the offset frequency. Find the approximate frequency offset in parts-per-millon (ppm).
* Repeat the above until the magnitude of the signal is within range and its frequency is centered. 
* Record the SDR gain and the ppm shift. You will need to use it later.

Use the frequency 919 MHz.

In [ ]:
# set up SDR
fs_sdr = 240000
fc = ??? # set your frequency
ppm = ??? # set your ppm! (Hint: start with 1)
gain = ??? # set your SDR gain (Hint: 0 - 50)

sdr = RtlSdr()
sdr.sample_rate = fs_sdr
sdr.gain = gain
sdr.center_freq = fc
sdr.freq_correction = ppm

R = Radio()
R.freq_tx = 919000

# start transmitting
R.NFMtx()
time.sleep(0.5)
y = sdr.read_samples(fs_sdr)

# stop transmitting
R.close()
sdr.close()

In [ ]:
# code to plot amplitude and compute frequency here:


# set correction values
ppmcalib = int(ppm - np.round(f0/fc*1e6))
gaincalib = gain

print('shift in Hz:', f0)
print('shift in ppm:', ppmcalib)

Now that the SDR frequency and gain are calibrated, let's do the calibration of the audio level to the radio.

#### Task 1

* Generate a 4 second tone at 2200 Hz. The tone amplitude should vary linearly from 0 to 2 throughout the 4 seconds.
* Add 250 ms worth of zeros at the beginning of the array.
* Transmit the signal using the radio and simultaneously receive for 5 seconds using the SDR.

In [ ]:
# generate the tone


# set up SDR
fs_sdr = 240000
fc = 919.000e6 # set your frequency!

sdr = RtlSdr()
sdr.sample_rate = fs_sdr
sdr.gain = gaincalib
sdr.center_freq = fc
sdr.set_freq_correction(ppmcalib)

# start transmitting
R.NFMtx()
time.sleep(0.5)
sd.play(sig, samplerate=fs_se, device=loop1_idx, blocking=False) # play samples to radio

# read samples from SDR
y = sdr.read_samples(fs_sdr*5)

# stop transmitting when done
R.close()
sdr.close()

#### Task 2

* Plot the magnitude of the received signal, and pick a threshold to crop the samples corresponding to the transmission. 
* Again, make sure the amplitude of the signal is > 0.25 and < 0.75. If not, change the gain or move the SDR antenna away from the radio.
* Crop the signal to the transmission part -- make sure you have > 2 seconds of data.

In [ ]:
# your code here:


#### Task 3

* Plot the spectrogram. Make sure the signal is close to the center frequency. If not, adjust the frequency correction ppm accordingly. 
* Can you see the bandwidth increasing and then leveling? Why is that happening?

In [ ]:
# your code here:


tt, ff, xmf = myspectrogram_hann_ovlp(y_d, 256, fs_sdr, 0, dbf=20)

#### Task 4
* Lowpass filter with a bandwidth of 15 kHz.
* Downsample by 10 to 24 kHz effective sampling rate.
* FM demodulate using the approach in Lab 3.
* Plot the spectrogram of the demodulated signal.

In [ ]:
# your code here:


tt, ff, xmf = myspectrogram_hann_ovlp(y_df, 256, fs_sdr/10, 0, dbf=20)

#### Task 5

To see envelope of the transmitted tone: 
* Create a narrow single-sideband bandpass filter by complex modulating a Hann window (`signal.hann`) of length 513 to a center frequency around 2200 Hz
* Filter the demodulated signal and display its magnitude (use `mode='same'` to compensate for the filter delay).
* You should see a linear ramp that starts tapering near the maximum and then becomes flat. Find the time in seconds it took from the beginning of the ramp until just before it starts to roll off. Divide that value by 2 and you've got yourself the maximum amplitude that results in a linear response!

#### Save the value of the maximum amplitude that is linear! (HINT: for RPiTX, the linear regime should be 0-1, hence a linear ramp for 2 seconds and then roll off.)

In [ ]:
%matplotlib notebook
%matplotlib notebook

# your code here:


In [ ]:
%matplotlib inline
%matplotlib inline

#### Measuring the Frequency Response of the Radio's Bandpass Audio Filter

As mentioned earlier, in traditional handheld radios, the audio input to the radio is filtered by a bandpass filter. They also emphasize the high frequencies with a filter of approximately 6 dB per decade. We simulate this by using CSDR to filter the audio before sending to RPiTX.

If the filter is unknown, it is possible to estimate its frequency response by transmitting a known signal and measuring the result. Here, we  will use a chirp signal to estimate the magnitude frequency response. We will transmit with the RPiTX radio and receive using the SDR.

#### Task 6

* Generate a chirp from 20 Hz to 8 kHz over 2 seconds.
* Transmit using the radio, and record using the SDR (for 3 seconds).
* Crop based on amplitude, filter, decimate, and FM demodulate.
* Plot the spectrogram and the magnitude frequency response of the result.

#### IMPORTANT --  the ppm calibration only lasts for a while. Its results can vary significantly (I observed -20 ppm to 20 ppm). If you are having poor demodulation results, perform the calibration again before proceeding.

In [ ]:
def genChirpPulse(Npulse, f0, f1, fs):
    #     Function generates a complex chirp pulse
    #     Inputs: Npulse - pulse length in samples
    #             f0     - start frequency of chirp
    #             f1     - end frequency of chirp
    #             fs     - sampling frequency
    
    t1 = r_[0.0:Npulse]/fs
    Tpulse = Npulse / fs
    f_of_t = f0 + t1 / Tpulse * (f1 - f0)
    phi_of_t = 2*pi * np.cumsum(f_of_t)/fs
    pulse = np.exp(1j*phi_of_t)
    return pulse

In [ ]:
# generate the signal


# set up SDR


# start transmitting


# stop transmitting when done


In [ ]:
# downsample and demodulate the FM signal


In [ ]:
# plot the spectrogram and the average power spectrum 


Another way of estimating a frequency response is to transmit white noise, which has uniform energy throughout the spectrum. Its name is derived from the fact that light with equal contributions from every visible frequency appears white (optics!!!!!).

#### Task 7

* Generate 4 seconds (at 48 kHz sampling rate) of white uniform noise between -1 and 1 using `np.random.rand`.
* Scale the amplitude to the maximum linear amplitude value you found previously.
* Transmit using the radio, and record using the SDR.
* Crop based on amplitude, filter, decimate, and FM demodulate.
* Plot the spectrogram.

In order to display a non-noisy spectrum, we will need to compute an average power spectrum. Use the function `avgPS` to do so.

* Use a window size of 128 and plot the square root of the result for the positive frequencies. 

In [ ]:
# generate the signal


# set up SDR


# start transmitting


# stop transmitting when done


# downsample and demodulate


# plot spectrogram and average power spectrum



### Transmitting your callsign in Morse code

The next step is to see if you can transmit something more meaningful. If you are going to transmit for the first time using a computer, you might as well transmit your callsign in Morse code!

Morse code is composed of dots (notated . and pronounced dit) and dashes (notated - and pronounced dah). The timing is relative to a dit duration which is one unit long. A dah is three units long. The gap between dots and dashes within a character is one unit. A short gap between letters is three units and a gap between words is seven units.

A dictionary of Morse code is provided.

#### Task 8

* Implement a function `sig = text2Morse(text, fc, fs,dt)`. The function will take a string and convert it to a tone signal of the Morse code of the text. The function will also take 'fc' the frequency of the tones (800-900 Hz sounds nice), 'fs' the sampling frequency, and 'dt' the Morse unit time (hence the speed, 50-75 ms recommended).
* Transmit your call sign! You can use this function to identify yourself before, during, and after a transmission from now on.
* Validate the code by capturing a spectrogram using the SDR.

You can also play the sound of your recording!!!

In [ ]:
def text2Morse(text,fc,fs,dt):
    CODE = {'A': '.-',     'B': '-...',   'C': '-.-.', 
        'D': '-..',    'E': '.',      'F': '..-.',
        'G': '--.',    'H': '....',   'I': '..',
        'J': '.---',   'K': '-.-',    'L': '.-..',
        'M': '--',     'N': '-.',     'O': '---',
        'P': '.--.',   'Q': '--.-',   'R': '.-.',
        'S': '...',    'T': '-',      'U': '..-',
        'V': '...-',   'W': '.--',    'X': '-..-',
        'Y': '-.--',   'Z': '--..',
        
        '0': '-----',  '1': '.----',  '2': '..---',
        '3': '...--',  '4': '....-',  '5': '.....',
        '6': '-....',  '7': '--...',  '8': '---..',
        '9': '----.',

        ' ': ' ', "'": '.----.', '(': '-.--.-',  ')': '-.--.-',
        ',': '--..--', '-': '-....-', '.': '.-.-.-',
        '/': '-..-.',   ':': '---...', ';': '-.-.-.',
        '?': '..--..', '_': '..--.-'
        }
    
    Ndit = np.int32(1.0*fs*dt)
    Ndah = 3*Ndit
    
    sdit = np.sin(2*pi*fc*r_[0.0:Ndit]/fs)
    sdah = np.sin(2*pi*fc*r_[0.0:Ndah]/fs)
    
    # convert to dit dah
    mrs = ""
    for char in text:
        mrs = mrs + CODE[char.upper()] + "*"
    
    sig = zeros(1)
    for char in mrs:
        if char == " ":
            sig = np.concatenate((sig,zeros(Ndit*7)))
        if char == "*":
            sig = np.concatenate((sig,zeros(Ndit*3)))
        if char == ".":
            sig = np.concatenate((sig,sdit,zeros(Ndit)))
        if char == "-":
            sig = np.concatenate((sig,sdah,zeros(Ndit)))
    return sig

In [ ]:
fs_sdr = 240000
fc = 919.000e6

sdr = RtlSdr()
sdr.sample_rate = fs_sdr
sdr.gain = gaincalib
sdr.center_freq = fc
sdr.set_freq_correction(ppmcalib)

R = Radio()
R.freq_tx = 919000

fs_se = 48000
callsign = text2Morse(YOUR CALLSIGN e.g. "KN6IFE", 850, fs_se, 75e-3)
callsign = np.concatenate((np.zeros(fs_se//10),callsign))                    
T = len(callsign) / fs_se # time to transmit

R.NFMtx()
time.sleep(0.1)
sd.play(callsign, samplerate=fs_se, device=loop1_idx, blocking=False)

y = sdr.read_samples(int(fs_sdr*(T+1)))

sdr.close()
R.close()

In [ ]:
# downsample and demodulate

y_df = ???

y_play = signal.resample( y_df/max(abs(y_df))*0.9, len(y_df)*2 )

In [ ]:
sd.play(y_play, samplerate=fs_se, device=builtin_idx, blocking=True)

In [ ]:
tt, ff, xmf = myspectrogram_hann_ovlp(y_df, 256, fs_sdr/10, 0, dbf=30)